# Memory
- 기본 개요
    - 랭체인에는 5가지 정도의 저장방식이 있음
    - 각자 사용방법/ 장단점이 다름
    - 챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없음
    - 유저가 자신의 이름을 말하거나 이전 질문에 이어지는 질문을 해도 메모리, 즉 기역력이 없기때문에 대화를 이해할 수 있는 능력이 없음
    - openAI에서 제공하는 기본 API는 랭체인없이 사용할 수 있지만 메모리를지원하지 않음
    - 즉 어떤 모델에 대화를 걸어더 나중에 다 까먹게됨
    - 챗gpt에는 메모리가 존재하고 실제로 어떤 사람과 대화하는 느낌을 가지게함
    - 챗봇이 이전의 대화나 질문을 기억하고 답할 수 있기때문임
    - 따라서 이번 강의에서는 이런 각 메모리의 종류와 차이점을 보고, 랭체인에 메모리를 탑재하는 방법을 알아봄
- 기본 흐름
    - 모든 메모리는 save_context, load_memory_variables, input, output 함수를 가지고 있음, 메모리 종류만 다를뿐 API는 동일함
- return_message
    - 메모리를 우선적으로 만들고, 챗 모델을 위한것 인지 아닌지 선택하고, 챗모델을 위한게 아니라면 return_message=False로 하거나 빼도 됨
    - 그럼 history가 문자열로만 표시될 것
    - 쳇모델을 사용하고싶다면 True로 바꾸면 챗 모델이 사용할 수 있는 형태로 출력됨


## 5.1 Conversation Buffer Memory
- 단순히 이전 대화내용을 기억하는 메모리
- 장점 : 가장 이해하기 쉬운 메모리임
- 단점 : 대화내용이 길어질수록 메모리도 커지니까 비효율적임


- 모델 자체에는 메모리가 없음
  - 우리가 모델에 요청을 보낼때 이전 대화 기록 전체를 같이 보내줘야함
  - 그래야 모델이 전에 일어난 대화를 보고 이해할 수 있음
  - 유저와 AI 대화가 길어질 수 록 우리가 매번 모델에 보내야할 기록이 길어진것을 의미하며 상당히 비효율적이고 돈이 많이 듬



In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({}) # history로 대화한 내용을 불러움



{'history': 'Human: Hi\nAI: 뭐하니?'}

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({}) # history로 대화한 내용을 불러움
# {'history': [HumanMessage(content='Hi'), AIMessage(content='뭐하니?')]}

{'history': [HumanMessage(content='Hi'), AIMessage(content='뭐하니?')]}

In [4]:
# 이 메모리는 대화를 저장하고 불러오는 기능을 가지기 떄문에 메모리에 대화내용이 계속 누적시킴

memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?')]}

In [5]:
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?')]}

In [ ]:
# 대화가 길어질 수 록 수많은 내용이 계속 쌓이기때문에 비효율적이라, 좀 더 효율적으로 메모리, 대화 기록을 최소화하는 메모리를 확인할 것


## 5.2 ConversationBufferWindowMemory
- k=4 처럼 메모리 개수를 지정하여 모든 대화를 저장하지 않고 일부만 저장하도록 함
- 최신 것만 기억하는 것이 단점이지만 비용이 효율적일 수 있음

In [11]:
from langchain.memory import ConversationBufferWindowMemory

# 대화의 특정 부분만을 저장하는 메모리
# 예: 최근 5개만 저장하는 경우, 최근 5개만 저장하고 나머지는 삭제됨
# 메모리를 특정 크기로 유지할 수 있고, 모든 대화를 저장하지 않아도 됨
# 단점은 챗봇이 최근 대화만 기억함

memory = ConversationBufferWindowMemory(
    return_messages=True, 
    k = 4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    

add_message(1,1)

In [12]:
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

In [13]:
memory.load_memory_variables({})
# k=4이기 때문에 1이 삭제됨/ 최신 4개만 저장됨


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [14]:
add_message(6,6)
memory.load_memory_variables({}) # 최신 4개만 저장되어 3~6까지만 저장/ 단점은 챗봇이 최근 대화만 기억함

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}

## 5.3-1 ConverstaionSummaryMemory
- 메세지를 그대로 저장하는것이 아니라 conversation 자체를 요약하는것
- llm을 필요로함
- 매우 긴 conversation이 있는 경우 유용함
- 초반에는 이전보다 많은 토큰고 저장공간을 차지하지만 메세지가 많아질수록 도움이 됨
- 요약하는 것이 토큰 양도 줄면서 훨씬 나은 방법이됨
- 저장하는 것보다 COnversation을 사용하여 요약하는것이 훨씬 나음


In [15]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryMemory(llm = llm) 

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("안녕 여기는 서울입니다.","서울의 날씨는 어떠니?")


In [16]:
add_message("여기는 광주입니다.","광주가고싶다")

In [18]:
print(get_history())
""" 문장을 요약해서 저장해둠/ 내가 입력한 것 보다는 처음에 더 맣은 저장공간을 사용하여 토큰을 사용하지만 대화가 길어질 수록 summary가 각 메세지보다 더 효율적임
{'history': 'The human greets in Korean and mentions they are in Seoul. The AI asks about the weather in Seoul. The human responds that they are in Gwangju. The AI expresses a desire to visit Gwangju.'}
"""

{'history': 'The human greets in Korean and mentions they are in Seoul. The AI asks about the weather in Seoul. The human responds that they are in Gwangju. The AI expresses a desire to visit Gwangju.'}


## 5.3-2 ConversationSummaryBufferMemory
- Conversation Buffer Memory와 Conversation Summary Memory의 결합임
- 메모리에 보내온 메세지 수를 제한
- 오래된 메세지들을 요약함
- 가장 최근의 상호작용을 계속 추적하며
- 가장 최근/ 가장 오래전에 받은 이야기를 모두 추적하고있다는 것

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_tokens = 10 # 최대 토큰 수/ 메세지들이 요약되기 전까지의 토큰 수 
) 

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("안녕 여기는 서울입니다.","서울의 날씨는 어떠니?")
